In [6]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.preprocessing import LabelEncoder

In [9]:
# Null값 처리
def fillna(df: pd.DataFrame) -> pd.DataFrame:
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N',inplace= True)
    df['Embarked'].fillna('N',inplace= True)
    return df

# 불필요한 칼럼 제거
def drop_featrues(df: pd.DataFrame) -> pd.DataFrame:
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 문자열 칼럼 숫자롤 바꿔주기(인코딩)
def format_features(df: pd.DataFrame) -> pd.DataFrame:
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    
    for feature in features:
        label = LabelEncoder()
        label = label.fit(df[feature])
        df[feature] = label.transform(df[feature])
        
    return df

# 위의 3함수 묶어서 실행
def transform_features(df: pd.DataFrame) -> pd.DataFrame:
    df = fillna(df)
    df = drop_featrues(df)
    df = format_features(df)
    return df


In [10]:
titanic_df = pd.read_csv("../data/titanic_train.csv")

# 타겟 데이터 분리
y_titanic_df = titanic_df['Survived']
# 타겟 데이터 원본에서 제거
X_titanic_df = titanic_df.drop('Survived', axis=1, inplace=False)
# 데이터 가공
X_titanic_df = transform_features(X_titanic_df)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11)

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [33]:
# 결정트리, 랜덤 포레스트, 로지스틱 회귀 이용해보자 !!
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

In [34]:
# 결정트리
dt_clf.fit(X_train, y_train) # 데이터 학습
dt_pred = dt_clf.predict(X_test) # 테스트 데이터 넣어서 예측
print('결정트리 정확도: {0:.4f}'.format(accuracy_score(y_test,dt_pred))) # 실제 데이터와 얼마나 일치하는지 확인(정확도 확인)

# 랜덤 포레스트
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
print('랜덤 포레스트 정확도:  {0:.4f}'.format(accuracy_score(y_test,rf_pred))) 

# 로지스틱 회귀
lr_clf.fit(X_train, y_train)
lr_pred = rf_clf.predict(X_test)
print('로지스틱 회귀 정확도:  {0:.4f}'.format(accuracy_score(y_test,lr_pred))) 

결정트리 정확도: 0.7877
랜덤 포레스트 정확도:  0.8547
로지스틱 회귀 정확도:  0.8547


In [46]:
from sklearn.model_selection import KFold

# 교차검증
def exec_kfold(clf, folds = 5):
    kfold = KFold(n_splits=folds)
    scores = []

    # 5번 돌리기 위해 for문 이용
    for i, (train_index, test_index) in enumerate(kfold.split(X_titanic_df)):
        X_train, X_test =  X_titanic_df.values[train_index], X_titanic_df.values[test_index]
        y_train, y_test =  y_titanic_df.values[train_index], y_titanic_df.values[test_index]
        
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, pred)
        scores.append(accuracy)
        print('교차검증 {0}차의 정확도: {1:.4f}'.format(i+1, accuracy))
    
    print("평균 정확도: {0:.4f}".format(np.mean(scores)))

exec_kfold(dt_clf)
exec_kfold(rf_clf)
exec_kfold(lr_clf)
    

교차검증 1차의 정확도: 0.7542
교차검증 2차의 정확도: 0.7809
교차검증 3차의 정확도: 0.7865
교차검증 4차의 정확도: 0.7697
교차검증 5차의 정확도: 0.8202
평균 정확도: 0.7823
교차검증 1차의 정확도: 0.7933
교차검증 2차의 정확도: 0.8090
교차검증 3차의 정확도: 0.8371
교차검증 4차의 정확도: 0.7753
교차검증 5차의 정확도: 0.8596
평균 정확도: 0.8148
교차검증 1차의 정확도: 0.7933
교차검증 2차의 정확도: 0.7921
교차검증 3차의 정확도: 0.7753
교차검증 4차의 정확도: 0.7472
교차검증 5차의 정확도: 0.8427
평균 정확도: 0.7901


In [49]:
# 위의 kfold보다 교차검증을 훨씬 간단하게 할 수 있다.
from sklearn.model_selection import cross_val_score

def exec_cross(clf, cv =5):
    scores = cross_val_score(clf, X_titanic_df, y_titanic_df, cv=5)    
    for i, accuracy in enumerate(scores):
        print("교차검증 {0}차의 정확도: {1:.4f}.".format(i + 1, accuracy))
    print("평균 정확도: {0:.4f}".format(np.mean(scores)))

exec_cross(dt_clf)
exec_cross(rf_clf)
exec_cross(lr_clf)

교차검증 1차의 정확도: 0.7430.
교차검증 2차의 정확도: 0.7753.
교차검증 3차의 정확도: 0.7921.
교차검증 4차의 정확도: 0.7865.
교차검증 5차의 정확도: 0.8427.
평균 정확도: 0.7879
교차검증 1차의 정확도: 0.7933.
교차검증 2차의 정확도: 0.7978.
교차검증 3차의 정확도: 0.8483.
교차검증 4차의 정확도: 0.7640.
교차검증 5차의 정확도: 0.8652.
평균 정확도: 0.8137
교차검증 1차의 정확도: 0.7877.
교차검증 2차의 정확도: 0.7921.
교차검증 3차의 정확도: 0.7753.
교차검증 4차의 정확도: 0.7640.
교차검증 5차의 정확도: 0.8202.
평균 정확도: 0.7879


In [55]:
from sklearn.model_selection import GridSearchCV

# 36번 학습시키고 예측 수행 (더 많이 학습시키니까 정확해진다..)
parameters = {'max_depth': [2,3,5,10], 'min_samples_split': [2,3,5], 'min_samples_leaf': [1,5,8]}

# 결정트리
grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=5)
grid_dclf.fit(X_train, y_train)
print("최적 하이퍼 파라미터:", grid_dclf.best_params_)
print("최고 정확도: {0:.4f}".format(grid_dclf.best_score_))

# 랜덤 포레스트 (왜 이렇게 오래걸리지?)
# grid_dclf = GridSearchCV(rf_clf, param_grid=parameters, scoring='accuracy', cv=5)
# grid_dclf.fit(X_train, y_train)
# print("최적 하이퍼 파라미터:", grid_dclf.best_params_)
# print("최고 정확도: {0:.4f}".format(grid_dclf.best_score_))

# 로지스틱 회귀 (왜 안되지??)
# grid_dclf = GridSearchCV(lr_clf, param_grid=parameters, scoring='accuracy', cv=5)
# grid_dclf.fit(X_train, y_train)
# print("최적 하이퍼 파라미터:", grid_dclf.best_params_)
# print("최고 정확도: {0:.4f}".format(grid_dclf.best_score_))

최적 하이퍼 파라미터: {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
최고 정확도: 0.7992
